<a href="https://colab.research.google.com/github/karanamrahul/END-Course-NLP/blob/main/Week_10_Assignment_Part_1(Align_and_translate).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assigment - 10 

Seq2Seq Model with attention mechanism over packed padded sentence sequences.

Dataset-We are using Multi30K dataset and our model will be translating from English to German.

Resources to follow in order to understand this model better are given below:

1.   [Attention is all you need](https://arxiv.org/abs/1706.03762)
2.   [Neural Machine Translation by Jointly Learning to Align and Translate](https://arxiv.org/pdf/1409.0473.pdf)
1.   [Packed Padded Sequences, Masking, Inference and BLEU](https://github.com/bentrevett/pytorch-seq2seq/blob/master/4%20-%20Packed%20Padded%20Sequences%2C%20Masking%2C%20Inference%20and%20BLEU.ipynb)
2.   [Pytorch tutorial on language translation](https://pytorch.org/tutorials/beginner/torchtext_translation_tutorial.html)






In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


from torchtext.datasets import Multi30k
from torchtext.data import Field,BucketIterator

import spacy
import numpy as np
import random 
import math
import time


SEED=1111

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [14]:
#Load german and english data from spacy
!python -m spacy download en
!python -m spacy download de


✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [15]:
spacy_de=spacy.load('de')
spacy_en=spacy.load('en')

In [16]:
# These two function will tokenize german english text into a list of strings

def tokenize_de(text):

  return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):

  return [tok.text for tok in spacy_en.tokenizer(text)]



In [17]:

SRC = Field(tokenize = tokenize_de, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [18]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), 
                                                    fields = (SRC, TRG))

In [19]:
#Build the vocab
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [21]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

In [64]:
class Encoder(nn.Module):
  def __init__(self,input_dim,emb_dim,enc_hid_dim,dec_hid_dim,dropout):
    super().__init__()

    self.embedding = nn.Embedding(input_dim, emb_dim)    
    self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True) 
    self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
    self.dropout = nn.Dropout(dropout)   
    

  def forward(self,src):
    #src=[source length,batch size]
    embedding=self.dropout(self.embedding(src))
    # embedding=[source length,batch size,emb dim]
    outputs,hidden=self.rnn(embedding)
    # outputs = [source length,batch size, enc_hid_dim*num_directions]
    # hidden = [num_layers*num_directions,batch size,enc_hid_dim]
    #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
    hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
    #hidden [-2, :, : ] is the last of the forwards RNN 
    #hidden [-1, :, : ] is the last of the backwards RNN

    #outputs=[source length,batch size,enc_hid_dim*2]
    #hidden=[batch size,dec_hid_dim]

    return outputs,hidden



In [23]:
## Example of output,embedded and hidden size
# enc=Encoder(input_dim=32,emb_dim=256,enc_hid_dim=512,dec_hid_dim=512,dropout=0.1)
# emb,out,hid,hid_cat=enc.forward(torch.zeros(12,32).to(torch.int64))

# emb.size()=torch.Size([12, 32, 256]),
# out.size()=torch.Size([12, 32, 1024]),
# hid.size()=torch.Size([2, 32, 512]))
# hid_cat.size()=torch.Size([32,512])

In [65]:
class Attention(nn.Module):
  def __init__(self,enc_hid_dim,dec_hid_dim):
    super().__init__()
    #The below method is also called as energy with respect to paper.
    self.attn=nn.Linear(enc_hid_dim*2+dec_hid_dim,dec_hid_dim)
    self.v=nn.Linear(dec_hid_dim,1,bias=False)

  def forward(self,hidden,encoder_outputs):

    src_len = encoder_outputs.shape[0]
    #encoder_outputs=[src_len,batch_size,enc_hid_dim*2]
    #hidden=[batch_size,dec_hid_dim]
        
    #repeat decoder hidden state src_len times
    hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        
    encoder_outputs = encoder_outputs.permute(1, 0, 2)
    # After the above two operations
    #encoder_outputs=[batch_size,src_len,enc_dim*2]
    #hidden=[batch_size,src_len,dec_dim]

    energy=torch.tanh(self.attn(torch.cat((hidden,encoder_outputs),dim=2)))
    #energy=[batch_size,src_len,dec_hid_dim]

    attention=self.v(energy).squeeze(2)
    #attention=[batch_size,src_len]

    return F.softmax(attention,dim=1)








In [88]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs)
                
        #a = [batch size, src len]
        
        a = a.unsqueeze(1)
        
        #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
       
        
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden.squeeze(0)

In [89]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)
                
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and all encoder hidden states
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [90]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

In [91]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7855, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(5893, 256)
    (rnn): GRU(1280, 512)
    (fc_out): Linear(in_features=1792, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [92]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 20,518,917 trainable parameters


In [93]:
optimizer = optim.Adam(model.parameters())

In [94]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [95]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [96]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [97]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [98]:
N_EPOCHS = 20
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 1m 26s
	Train Loss: 5.057 | Train PPL: 157.083
	 Val. Loss: 4.925 |  Val. PPL: 137.700
Epoch: 02 | Time: 1m 25s
	Train Loss: 4.110 | Train PPL:  60.957
	 Val. Loss: 4.294 |  Val. PPL:  73.291
Epoch: 03 | Time: 1m 25s
	Train Loss: 3.414 | Train PPL:  30.394
	 Val. Loss: 3.645 |  Val. PPL:  38.272
Epoch: 04 | Time: 1m 26s
	Train Loss: 2.889 | Train PPL:  17.967
	 Val. Loss: 3.330 |  Val. PPL:  27.926
Epoch: 05 | Time: 1m 25s
	Train Loss: 2.487 | Train PPL:  12.020
	 Val. Loss: 3.295 |  Val. PPL:  26.981
Epoch: 06 | Time: 1m 26s
	Train Loss: 2.189 | Train PPL:   8.930
	 Val. Loss: 3.175 |  Val. PPL:  23.933
Epoch: 07 | Time: 1m 26s
	Train Loss: 1.928 | Train PPL:   6.873
	 Val. Loss: 3.275 |  Val. PPL:  26.450
Epoch: 08 | Time: 1m 26s
	Train Loss: 1.745 | Train PPL:   5.727
	 Val. Loss: 3.283 |  Val. PPL:  26.658
Epoch: 09 | Time: 1m 26s
	Train Loss: 1.597 | Train PPL:   4.940
	 Val. Loss: 3.337 |  Val. PPL:  28.139
Epoch: 10 | Time: 1m 26s
	Train Loss: 1.448 | Train PPL

In [99]:
model.load_state_dict(torch.load('tut3-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.182 | Test PPL:  24.096 |
